#### This notebook is used to find similar questions from SO given a question from SO.

In [1]:
import pandas as pd
import numpy as np

import nltk
import string
import os
import collections

import random
import datetime
import json
import heapq
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer

# Data

In [2]:

post_df = pd.read_csv('all_posts.csv', sep = "\t")
post_df.head()

,Id,PostTypeId,ParentId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount
0,1,1,NaN,15.0,2010-07-19T19:12:12.510,36,2577.0,How should I elicit prior distributions from e...,8.0,2010-09-15T21:08:26.077,Eliciting priors from experts,<bayesian><prior><elicitation>,5.0,1,23.0
1,2,1,NaN,59.0,2010-07-19T19:12:57.157,29,23368.0,In many different statistical methods there is...,24.0,2016-06-27T06:44:40.147,What is normality?,<distributions><normality>,7.0,1,10.0
2,3,1,NaN,5.0,2010-07-19T19:13:28.577,66,5792.0,What are some valuable Statistical Analysis op...,18.0,2013-05-27T14:48:36.927,What are some valuable Statistical Analysis op...,<software><open-source>,19.0,4,39.0
3,4,1,NaN,135.0,2010-07-19T19:13:31.617,17,26414.0,I have two groups of data. Each with a differ...,23.0,2010-09-08T03:00:19.690,Assessing the significance of differences in d...,<distributions><statistical-significance>,5.0,2,5.0
4,5,2,3.0,NaN,2010-07-19T19:14:43.050,87,NaN,The R-project\n\nhttp://www.r-project.org/\n\n...,23.0,2010-07-19T19:21:15.063,NaN,NaN,NaN,3,NaN


In [3]:
Q_df = post_df.loc[post_df['PostTypeId'] == 1][['Id', 'Title','Body']]

Q_df.head()

,Id,Title,Body
0,1,Eliciting priors from experts,How should I elicit prior distributions from e...
1,2,What is normality?,In many different statistical methods there is...
2,3,What are some valuable Statistical Analysis op...,What are some valuable Statistical Analysis op...
3,4,Assessing the significance of differences in d...,I have two groups of data. Each with a differ...
5,6,The Two Cultures: statistics vs. machine learn...,"Last year, I read a blog post from Brendan O'C..."


In [6]:
size = 200
min_count = 2
window = 6
key = 'word2vector_model_question_answer_' + str(int(size))+"_"+str(int(window))+"_"+str(int(min_count))
key

'word2vector_model_question_answer_200_6_2'

In [7]:
questions_vec_df = pd.read_csv('doc_vec/questions_vec_weighted.csv')
questions_vec_df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,0,-0.274664,-0.197037,1.621728,-0.237445,-0.288944,0.843277,0.617973,1.499649,0.842379,...,-0.057703,-0.040805,0.639898,0.234292,0.024925,0.118979,-0.794148,-0.566620,0.138416,-0.548374
1,1,-0.009377,-0.493120,2.030924,-0.442848,-0.290093,0.380474,1.293419,1.768064,1.919836,...,0.605748,0.176924,0.618500,-0.351151,-0.435653,0.027308,-0.069966,0.080411,1.921239,-0.199846
2,2,0.144012,-0.184814,0.698455,-0.249389,-0.030401,-0.010584,0.131361,0.616810,0.120255,...,0.412973,0.268411,-0.134549,-0.435909,-0.054599,0.377649,0.198194,-0.116156,0.011338,-0.133228
3,3,0.119089,0.075075,1.400657,-0.792461,-0.304917,0.148512,0.428406,1.037877,0.446540,...,0.973830,-0.436764,0.745695,-0.057611,0.056279,-0.127634,-0.005717,0.302014,0.462430,0.046998
4,4,0.217803,-0.220348,0.532312,-0.145882,-0.036074,0.122394,-0.028780,0.217096,0.266330,...,0.062220,0.167978,0.064264,-0.032044,-0.126803,0.170066,-0.218061,-0.247630,-0.101045,-0.194075


In [8]:
questions_vec_df = questions_vec_df.drop(questions_vec_df.columns[0], axis=1)

In [9]:
questions_vec_df.shape

(108954, 200)

## Cos similarity

In [11]:
from sklearn.metrics.pairwise import cosine_similarity


In [12]:
X =questions_vec_df.values

In [13]:

# Change chunk_size to control resource consumption and speed
# Higher chunk_size means more memory/RAM needed but also faster 
chunk_size = 500 
matrix_len = X.shape[0] # Not sparse numpy.ndarray

def similarity_cosine_by_chunk(start, end):
    if end > matrix_len:
        end = matrix_len
    return cosine_similarity(X=X[start:end], Y=X) # scikit-learn function
cosine_similarity_chunk = []
for chunk_start in range(0, 500, chunk_size):
     cosine_similarity_chunk.append(similarity_cosine_by_chunk(chunk_start, chunk_start+chunk_size))
    # Handle cosine_similarity_chunk  ( Write it to file_timestamp and close the file )
    # Do not open the same file again or you may end up with out of memory after few chunks 

In [14]:
c = cosine_similarity_chunk[0]

In [15]:
c.shape

(500, 108954)

In [16]:
for i in range(10):
    most_similar = c[i].argsort()[-4:][::-1]
    print()
    print()
    print("*************************question" + str(i) +"**************************")
    print(Q_df.iloc[i]['Body'])
    for j in most_similar:
        
        
        print("-------------------------------"+'related questions'+"------------------------")
        print(Q_df.iloc[j]['Body'])
        



*************************question0**************************
How should I elicit prior distributions from experts when fitting a Bayesian model?

-------------------------------related questions------------------------
How should I elicit prior distributions from experts when fitting a Bayesian model?

-------------------------------related questions------------------------
I have a question regarding model comparison using Bayes factors. In many cases, statisticians are interested on using a Bayesian approach with improper priors (for example some Jeffreys priors and reference priors). 

My question is, in those cases where the posterior distribution of the model parameters is well-defined, is it valid to compare models using Bayes factors under the use of improper priors?

As a simple example consider comparing a Normal model vs. a Logistic model with Jeffreys priors.

-------------------------------related questions------------------------
Bayesian methods have belief over the pri



*************************question7**************************
Many studies in the social sciences use Likert scales.  When is it appropriate to use Likert data as ordinal and when is it appropriate to use it as interval data?

-------------------------------related questions------------------------
Many studies in the social sciences use Likert scales.  When is it appropriate to use Likert data as ordinal and when is it appropriate to use it as interval data?

-------------------------------related questions------------------------
Can I use Likert scale data to do parametric statistical procedures such as linear regression and ANOVA? I've read that you can use Likert data in parametric analysis but in certain circumstances. Which are those? My independent variables are Likert items and my dependent variable is the mean score of a Likert scale. Also, I have 90 samples.

-------------------------------related questions------------------------
In educational research, some researchers s

# K means model

In [ ]:
X = questions_vec_weighted_df.values

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=100, random_state=0).fit(X)

In [ ]:
kmeans.labels_

In [ ]:
kmeans.predict(X)